In [1]:
from tensorboardX import SummaryWriter
from torch.autograd import Variable
from model import *
from torchvision import transforms
import util.loader.transform as tr
from loader import Data_Preprocess
from util.metrics import runningScore
from util.loss import *
from util.utils import *
from util.helpers import *

img_rows   = 240
img_cols   = 320
batch_size = 8
lr         = 1e-4

ttransforms = transforms.Compose([ tr.RandomHorizontalFlip(), 
                                   tr.Resize([img_rows, img_cols]),
                                   tr.ToTensor()])
tdataset = Data_Preprocess(db_root_dir='../data', train=True, transform=ttransforms)
tdataloader = torch.utils.data.DataLoader(tdataset, batch_size=batch_size, shuffle=True, num_workers=8)

vtransforms = transforms.Compose([ tr.Resize([img_rows, img_cols]),tr.ToTensor()])
vdataset = Data_Preprocess(db_root_dir='../data', train=False, transform=vtransforms)
vdataloader = torch.utils.data.DataLoader(vdataset, batch_size=batch_size, shuffle=False, num_workers=8)


# Setup Metrics
running_metrics = runningScore(pspnet_specs['n_classes'])

# setup Model
base_net = BaseNet()
class_net = ClassNet()

base_net.cuda()
class_net.cuda()

base_opt = torch.optim.Adam(base_net.parameters(), lr=lr,     betas=(0.5, 0.999))
class_opt = torch.optim.Adam(class_net.parameters(), lr=lr,   betas=(0.5, 0.999))

loss_fn = cross_entropy2d
len(tdataloader),len(vdataloader)

Done initializing train_seqs Dataset
Done initializing val_seqs Dataset


(468, 77)

In [2]:
writer = SummaryWriter('runs/train')
best_iou = 0.95
iter = 0
for epoch in range(50):
    base_net.train()
    class_net.train()
    
    for i, data in enumerate(tdataloader):
        images = Variable(data['image'].cuda())
        labels  = Variable(data['gt'].type(torch.LongTensor).cuda())
        
        poly_lr_scheduler(base_opt , lr, iter, lr_decay_iter=1, max_iter=1e+5)
        poly_lr_scheduler(class_opt, lr, iter, lr_decay_iter=1, max_iter=1e+5)
        iter += 1
        
        feature_map, inp = base_net(images)
        outputs = class_net(feature_map, inp)
        
        base_opt.zero_grad()
        class_opt.zero_grad()
        loss = loss_fn(input=outputs, target=labels)
        loss.backward()
        class_opt.step()
        base_opt.step()
        
        writer.add_scalar('Loss', loss.data[0], iter)
        if (i+1) % 100 == 0:
            print("Epoch [%d/%d] Loss: %.4f" % (epoch+1, 100, loss.data[0]))
            
    base_net.eval()
    class_net.eval()
    for i_val, data in enumerate(vdataloader):
        images = Variable(data['image'].cuda(),volatile=True)
        labels  = Variable(data['gt'].type(torch.LongTensor).cuda(),volatile=True)

        feature_map,inp = base_net(images)
        outputs = class_net(feature_map, inp)
        mask = outputs.data.max(1)[1].cpu()
        pred = mask.numpy()
        gt = labels.data.cpu().numpy()
        running_metrics.update(gt, pred)
        
    score, class_iou = running_metrics.get_scores()
    for k, v in score.items():
        print(k, v)
        if k == 'Mean IoU : \t':
            writer.add_scalar('IoU', v, epoch+1)
    if score['Mean IoU : \t'] >= best_iou:
        best_iou = score['Mean IoU : \t'] 
        torch.save(base_net.state_dict(), "weight/base_net.pkl")
        torch.save(class_net.state_dict(), "weight/class_net.pkl")
    running_metrics.reset() 

Epoch [1/100] Loss: 0.6249
Epoch [1/100] Loss: 0.5083
Epoch [1/100] Loss: 0.4803
Epoch [1/100] Loss: 0.3799
('FreqW Acc : \t', 0.9443461855068352)
('Overall Acc: \t', 0.9683576645457034)
('Mean Acc : \t', 0.9493406618029079)
('Mean IoU : \t', 0.8232103490658441)
Epoch [2/100] Loss: 0.3050
Epoch [2/100] Loss: 0.2396
Epoch [2/100] Loss: 0.2282
Epoch [2/100] Loss: 0.1931
('FreqW Acc : \t', 0.9290713219126063)
('Overall Acc: \t', 0.9556294471811713)
('Mean Acc : \t', 0.9411557956831231)
('Mean IoU : \t', 0.7903584872486452)
Epoch [3/100] Loss: 0.1351
Epoch [3/100] Loss: 0.1119
Epoch [3/100] Loss: 0.0948
Epoch [3/100] Loss: 0.1024
('FreqW Acc : \t', 0.9701788502934413)
('Overall Acc: \t', 0.9843683719896005)
('Mean Acc : \t', 0.9335820508218227)
('Mean IoU : \t', 0.8915674732220026)
Epoch [4/100] Loss: 0.0879
Epoch [4/100] Loss: 0.0772
Epoch [4/100] Loss: 0.0650
Epoch [4/100] Loss: 0.0667
('FreqW Acc : \t', 0.9650591788787576)
('Overall Acc: \t', 0.9804855979748222)
('Mean Acc : \t', 0.9673

Process Process-482:
Process Process-487:
Process Process-483:
Process Process-485:
Process Process-488:
Process Process-481:
Process Process-486:
Process Process-484:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", l

KeyboardInterrupt: 